In [1]:
import pandas as pd
import pickle as pk
from scipy.stats import poisson
import random
import csv


In [2]:
dict_table = pk.load(open('Base de datos\dict_table','rb'))
df_historico = pd.read_csv('Base de datos\clean_historico_partidos.csv')
df_fixture = pd.read_csv('Base de datos\main_fixture_copa_de_la_liga_2023.csv')


Calcular Team Strength

In [3]:
#Dividir df en df_home y df_away
df_historico
df_local = df_historico[['local','gol_local','gol_visitante']]
df_visitante = df_historico[['visitante','gol_visitante','gol_local']]

In [4]:
#Renombrar columnas
df_local = df_local.rename(columns={'local':'equipo','gol_local':'goles_anotados','gol_visitante':'goles_recibidos'})
df_visitante = df_visitante.rename(columns={'visitante':'equipo','gol_local':'goles_recibidos','gol_visitante':'goles_anotados'})

In [5]:
df_local_fuerza = df_local.groupby('equipo').mean()
df_visitante_fuerza = df_visitante.groupby('equipo').mean()

In [6]:
#Concatenar df_local y df_visitante, hacer group por team y calcular promedio
df_equipo_fuerza = pd.concat([df_local, df_visitante], ignore_index=True).groupby('equipo').mean()

In [7]:
df_equipo_fuerza = df_equipo_fuerza.sort_values('goles_anotados', ascending=False)

Simulacion Automatizada


In [8]:
df_campeones = pd.DataFrame()

df_campeones['local'] = None
df_campeones['visitante'] = None
df_campeones['campeon'] = None

In [9]:
df_campeones.to_csv('finales.csv', encoding='utf-8')

In [33]:

for i in range(500):
    
    
    #Carga de tablas
    dict_table = pk.load(open('Base de datos\dict_table','rb'))

    #Prediccion de puntos
    def simular_goles_en_partido(lambda_equipo):
        # generar un numero aleatorio entre 0 y 1
        rand_num = random.random()
        # inicializar variables en 0
        probabilidad_acumulada = num_goles = 0
        while True:
            # sumar la posibilidad de meter num_goles a probabilidad_acumulada
            probabilidad_acumulada += poisson.pmf(num_goles, lambda_equipo)
            # si probabilidad_acumulada supera a rand_num, devolver la cantidad de goles
            if probabilidad_acumulada >= rand_num:
                return num_goles
            # sino, aumentar la cantidad de goles e intentar de nuevo
            num_goles += 1
         
    #Predecir puntos
    def predecir_puntos(local, visitante, tabla_local, tabla_visitante):
        if local not in tabla_local.index or visitante not in tabla_visitante.index:
            assert False

        # goles de local = ( anotados locales + recibidos visitantes ) / 2
        goles_local  = simular_goles_en_partido(tabla_local.at[local,'goles_anotados'])
        goles_local += simular_goles_en_partido(tabla_visitante.at[visitante, 'goles_recibidos'])
        goles_local  = round(goles_local / 2)

        # goles de visitante = ( anotados visitantes + recibidos locales ) / 2
        goles_visitante  = simular_goles_en_partido(tabla_visitante.at[visitante,'goles_anotados'])
        goles_visitante += simular_goles_en_partido(tabla_local.at[local,'goles_recibidos'])
        goles_visitante  = round(goles_visitante / 2)
          
        #Almacenamos los resultados  
        with open ('resultados.csv', 'a', newline='', encoding='utf-8') as archivo_csv:
            fieldnames = ['local', 'gol_local', 'visitante', 'gol_visitante']
            writter = csv.DictWriter(archivo_csv, fieldnames=fieldnames)
            writter.writerow({'local': local, 'gol_local': goles_local, 'visitante': visitante, 'gol_visitante': goles_visitante}) 
          
        if goles_local == goles_visitante:
            #print(f'{local}: {goles_local} == {visitante}: {goles_visitante}')
            return (1, 1)
        elif goles_local > goles_visitante:
            #print(f'{local}: {goles_local} -> {visitante}: {goles_visitante}')
            return (3, 0)
        else:
            #print(f'{local}: {goles_local} <- {visitante}: {goles_visitante}')
            return (0, 3)

    #Predecir puntos local/visitante
    def predecir_puntos_locvis(local, visitante):
        return predecir_puntos(local, visitante, df_local_fuerza, df_visitante_fuerza)

    #Predecir puntos neutro
    def predecir_puntos_neutro(local, visitante):
        # evitar empates
        puntos_local, puntos_visitantes = 1, 1
        while puntos_local == 1 and  puntos_visitantes == 1:
            puntos_local, puntos_visitantes = predecir_puntos(local, visitante, df_equipo_fuerza, df_equipo_fuerza)
        return puntos_local, puntos_visitantes
    
    #Fixture fase de grupos
    df_fixture_grupos = df_fixture[:196].copy()
    df_fixture_cuartos = df_fixture[196:200].copy()
    df_fixture_semis = df_fixture[200:202].copy()
    df_fixture_final = df_fixture[202:].copy()
    
    #Correr todos los partidos de la fase de grupos y actualizar las tablas de cada grupo
    for grupo in dict_table:
        equipos_en_grupos = dict_table[grupo]['Equipo'].values
        df_fixture_2 = df_fixture_grupos[df_fixture_grupos['local'].isin(equipos_en_grupos)]      
        
        for index, row in df_fixture_2.iterrows():
            local, visitante = row['local'], row['visitante']
            puntos_local, puntos_visitantes = predecir_puntos_locvis(local, visitante)
            dict_table[grupo].loc[dict_table[grupo]['Equipo'] == local, 'Puntos'] += puntos_local
            dict_table[grupo].loc[dict_table[grupo]['Equipo'] == visitante, 'Puntos'] += puntos_visitantes 
            
    dict_table['Grupo A'] = dict_table['Grupo A'].sort_values('Puntos', ascending=False).reset_index()
    dict_table['Grupo B'] = dict_table['Grupo B'].sort_values('Puntos', ascending=False).reset_index()
    dict_table['Grupo A'] = dict_table['Grupo A'][['Equipo', 'Puntos']]
    dict_table['Grupo B'] = dict_table['Grupo B'][['Equipo', 'Puntos']]
    dict_table['Grupo A'] = dict_table['Grupo A'].round(0)
    dict_table['Grupo B'] = dict_table['Grupo B'].round(0)

    #Mostramos tablas
    dict_table_A = dict_table['Grupo A']
    dict_table_B = dict_table['Grupo B']
    
    #Almacenamos los datos
    with open(f"Resultados gigantes\dict_table_A_{i}","wb") as output:
        pk.dump(dict_table_A, output)
    
    with open(f"Resultados gigantes\dict_table_B_{i}","wb") as output:
        pk.dump(dict_table_B, output)
    
    #Asignamos 4tos
    for grupo in dict_table:
        primero_grupo = dict_table[grupo].loc[0, 'Equipo']
        segundo_grupo = dict_table[grupo].loc[2, 'Equipo']
        tercero_grupo = dict_table[grupo].loc[2, 'Equipo']
        cuarto_grupo = dict_table[grupo].loc[3, 'Equipo']
        df_fixture_cuartos.replace({f'1{grupo}':primero_grupo,f'2{grupo}':segundo_grupo, f'3{grupo}':tercero_grupo, f'4{grupo}':cuarto_grupo}, inplace=True)
    
    df_fixture_cuartos['Ganador'] = '?'
    
    #Ganador llave
    def ganador_llave(df_fixture_clasificados_cuartos):
        for index, row in df_fixture_clasificados_cuartos.iterrows():
            local, visitante = row['local'], row['visitante']
            puntos_local, puntos_visitantes = predecir_puntos_neutro(local, visitante)
            if puntos_local > puntos_visitantes:
                ganador = local
            else:
                ganador = visitante
            df_fixture_clasificados_cuartos.loc[index,'Ganador'] = ganador
            
            if ['final' in df_fixture_clasificados_cuartos['fecha'].values] == [True]:
                with open ('finales.csv', 'a', newline='', encoding='utf-8') as archivo_csv:
                    fieldnames = ['local', 'visitante', 'campeon']
                    writter = csv.DictWriter(archivo_csv, fieldnames=fieldnames)
                    writter.writerow({'local': local, 'visitante': visitante, 'campeon': ganador})
        return df_fixture_clasificados_cuartos
    
    #Ganador cuartos
    ganador_llave(df_fixture_cuartos)
    
    #Actualizar tabla
    def actualizar_tabla(df_fixture_ronda_1,df_fixture_ronda_2):
        for index, row in df_fixture_ronda_1.iterrows():
            ganador = df_fixture_ronda_1.loc[index,'Ganador']
            partido = df_fixture_ronda_1.loc[index, 'fecha']
            df_fixture_ronda_2.replace({f'ganador {partido}': ganador}, inplace=True)      
        df_fixture_ronda_2['Ganador'] = '?'
        return df_fixture_ronda_2    
    
    #Actualizar semis
    actualizar_tabla(df_fixture_cuartos,df_fixture_semis)
    
    #Ganador semis
    ganador_llave(df_fixture_semis)
    
    #Actualizar final
    actualizar_tabla(df_fixture_semis,df_fixture_final)
    
    #Ganador final
    campeon = ganador_llave(df_fixture_final)
    
print(campeon)

            local resultado           visitante  fecha             Ganador
202  Boca Juniors         -  Defensa y Justicia  final  Defensa y Justicia
